We will be working in the iPython / Jupyter notebook system. I like these because they are a form of literate programming in which we can mix textbook instruction and explanations with code that can also be run and edited.

The text and mathematics in the notebooks requires a little preliminary learning.

### Markdown

You can document your iPython notebooks by making some cells into **Markdown** cells. Markdown is a way of formatting text that is supposed to be almost as readable un-rendered as when it is tidied up. You might argue that it looks equally bad either way, but that's tough because the notebooks use it and that's how I want you to produce nice looking output to hand in as an assignment !

If you look at the **Markdown** cells as source code (by double-clicking on them) you will see how the raw text looks.

### Maths

In a browser, you can render beautiful equations using a javascript tool called **Mathjax** which is build into the iPython notebooks. 

You can build in symbols to your text such as $\pi$ and $\epsilon$ if you use the \$ signs to indicate where your equations begin and end, and you know enough $\LaTeX$ [try it here !](http://www.codecogs.com/latex/eqneditor.php) to get by.

Equations in 'display' mode are written like this

$$ e^{i\pi} + 1 = 0 $$

Go back to the rendered form of the cell by 'running' it.

### Links 

[Markdown Website](http://daringfireball.net/projects/markdown/)

[Mathjax Website](http://docs.mathjax.org)

### Datasets

We need to download some data for the examples. Some files are bundled with the notebooks but some are too large for the repository (and generally, that's not a place to keep anything which is not the primary target of revision management. 

#### Global Magnetic Data

Magnetic intensity data from [geomag.org](http://geomag.org/models/EMAG2/EMAG2_V2.tif)



In [1]:
## Bundle this up in a helper module 


import requests
import os
import time
import sys

def download_file(url, local_filename, expected_size):

    r = requests.get(url, stream=True)
             
    start_time = time.time()
    last_time = start_time
    datasize = 0
    
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1000000000): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                f.flush()
                datasize += len(chunk)
                if (time.time() - last_time) > 2.5:
                    print "{:.1f} Mb in {:.1f}s / {}".format(datasize / 1.0e6, time.time() - start_time, expected_size)
                    last_time = time.time()

    return 

import hashlib

def md5sum(filename, blocksize=65536):
    hash = hashlib.md5()
    with open(filename, "r+b") as f:
        for block in iter(lambda: f.read(blocksize), ""):
            hash.update(block)
    return hash.hexdigest()


def download_cached_file(location_url, local_file, expected_md5, expected_size="Unknown"):

    try:
        assert md5sum(local_file) == expected_md5
        print "Using cached file - {}".format(local_file)
        return(2)
    
    except (IOError, AssertionError) as error_info:
        # No file or the wrong file ... best go download it
        # print "Assertion failed - {}".format(sys.exc_info())
        
        try:
            mag_data_file = download_file(location_url, local_file, expected_size)
            print "Downloaded from {}".format( location_url )
            return(1)

        except:
            print "Unable to download {} [{}] ".format( location_url, sys.exc_info() )
            return(0)



In [6]:
# Resources required for the tutorial
# The checksum is to make sure any cached file is the right one.
# If the checksum is wrong the file gets downloaded anyway

resource_list = [
# Global magnetic data (120Mb)
{
 "local_file":"Resources/EMAG2_image_V2.tif",
 "md5":"20024c06fdabfa57e4f9efb4aabec198",
 "url":"http://geomag.org/models/EMAG2/EMAG2_image_V2.tif",
 "expected_size":"120Mb"
},
    
# Blue Marble Image Geotiff June 2004 (11Mb)  
{
 "local_file": "Resources/BlueMarbleNG-TB_2004-06-01_rgb_3600x1800.TIFF",
 "md5": '55e1399674d43a26353d84c114d7ff80',
 "url":"http://neo.sci.gsfc.nasa.gov/servlet/RenderData?si=526294&cs=rgb&format=TIFF&width=3600&height=1800",
 "expected_size":"11Mb"
},
    
    
# Blue Marble Image with Bathymetry png - December 2004 (11Mb)
{
 "local_file": "Resources/BlueMarbleNG-TB_2004-12-01_rgb_3600x1800.TIFF",
 "md5": '825da4b417ae19e24d2ea6db6cf8ad21',
 "url":"http://neo.sci.gsfc.nasa.gov/servlet/RenderData?si=526311&cs=rgb&format=TIFF&width=3600&height=1800",
 "expected_size":"11Mb"
},
    
# Etopo1 - color image (34M)
{
 "local_file": "Resources/color_etopo1_ice_low.tif.zip",
 "md5": '3f53d72e85393deb28874db0c76fbfcb',
 "url":"https://www.ngdc.noaa.gov/mgg/global/relief/ETOPO1/image/color_etopo1_ice_low.tif.zip",
 "expected_size":"34Mb"
}
]



In [7]:

for resource in resource_list:
    print "\nDownloading {:s}".format(resource["local_file"])
    download_cached_file(resource["url"], resource["local_file"], resource["md5"], resource["expected_size"] )



Using cached file - Resources/EMAG2_image_V2.tif

1.1 Mb in 2.5s / 2Mb
9.9 Mb in 5.1s / 2Mb
Downloaded from http://neo.sci.gsfc.nasa.gov/servlet/RenderData?si=526294&cs=rgb&format=TIFF&width=3600&height=1800

1.3 Mb in 2.5s / Unknown
10.0 Mb in 5.1s / Unknown
Downloaded from http://neo.sci.gsfc.nasa.gov/servlet/RenderData?si=526311&cs=rgb&format=TIFF&width=3600&height=1800

Using cached file - Resources/color_etopo1_ice_low.tif.zip


In [10]:
print md5sum("Resources/BlueMarbleNG-TB_2004-12-01_rgb_3600x1800.TIFF")

825da4b417ae19e24d2ea6db6cf8ad21


In [5]:
# Extract any files that were downloaded as zip archives

import zipfile
import glob

for zipped in glob.glob("Resources/*.zip"):
    with zipfile.ZipFile(zipped) as zf:
        zf.extractall("Resources")
        print "Unzipped {}".format(zipped)

Unzipped Resources/color_etopo1_ice_low.tif.zip


In [8]:
ls -l Resources


total 493824
-rw-r--r--  1 lmoresi  staff   10714734 14 Sep 15:39 BlueMarbleNG-TB_2004-06-01_rgb_3600x1800.TIFF
-rw-r--r--  1 lmoresi  staff   10854863 14 Sep 15:39 BlueMarbleNG-TB_2004-12-01_rgb_3600x1800.TIFF
-rw-r--r--  1 lmoresi  staff  134722670 14 Sep 15:14 EMAG2_image_V2.tif
-rw-r--r--  1 lmoresi  staff        181 14 Sep 14:28 README.md
-rw-r--r--  1 lmoresi  staff   43762944 14 Sep 15:30 color_etopo1_ice_low.tif
-rw-r--r--  1 lmoresi  staff   35751844 14 Sep 15:20 color_etopo1_ice_low.tif.zip
-rw-r--r--  1 lmoresi  staff    1941290 14 Sep 15:15 land_shallow_topo_2048.tif
-rw-r--r--  1 lmoresi  staff   15071572 14 Sep 15:15 world.topo.bathy.200409.3x5400x2700.png
